<a href="https://colab.research.google.com/github/nebulousman/labeling_reviews/blob/main/read_labeled_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Read Excel files with Labeled Reviews into DataFrame**
Dariush Moghadam

An earlier notebook (https://colab.research.google.com/drive/1xIfchmeOSgsQkkMRIRnWJiG6JSDlQs_e?authuser=1#scrollTo=oz9FSt-myoVj) selected random collection of consumer reviews of herbal products and distributed to volunteers for labeling.

This code reads the returned excel spreadsheets with labels and appends the Master Data containing returned/labeled reviews. Filename 'returned_labels_MASTER.csv' will then be processed and cleaned into a separate file for future analysis & addition to the original reviews file. As more labeled data is added to 'returned_labels_MASTER.csv' the cleaning process can be repeated.

This code also archives the labeled xlsx files that have been processed in a separate folder named 'processed'.

In [ ]:
#connect to google drive for dataset
# mount google drive for access to dataset previously added to the drive
from google.colab import drive
drive.mount('/content/drive')
#if you need to remount drive
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#imports
import pandas as pd
import numpy as np
import openpyxl
from pathlib import Path
import shutil
import os

In [ ]:
path = '/content/drive/My Drive/Analysis/labeling_reviews'

df = pd.concat([pd.read_excel(f, engine='openpyxl').dropna(how='all').dropna(axis=1,how='all')
                for f in Path(path+'/labeled_reviews').rglob('*.xlsx')])

In [ ]:
df.shape
#df_previous.shape
#df.dtypes

(100, 10)

In [ ]:
#open previously imported xlsx files and append new data
df_previous = pd.read_csv(path +'/master_files/returned_labels_MASTER.csv')

In [ ]:
#concatenate dataframes - stack on top of each other
df = pd.concat([df_previous,df], axis=0)

In [ ]:
#save file - note this overwrites previous version of returned_labels_MASTER.csv 
df.to_csv(path +'/master_files/returned_labels_MASTER.csv', index=False)

In [ ]:
#move processed files to new directory called 'processed'
source = path + '/labeled_reviews'
destination = path+'/master_files/labeled_reviews_originals'

file_names = [f for f in os.listdir(source) if f[-4:] == 'xlsx']
    
for file_name in file_names:
    shutil.move(os.path.join(source, file_name), destination)

In [ ]:
#remove rows where all labels are null - if the review was blank, not English or about something other than medicinal foods,
#volunteers were asked to skip which would result in no label assignment and all labels as null. We filter them out.
df = df.iloc[(df.iloc[:,2:9].dropna(how='all')).index,:] 

In [ ]:
#save file - note this overwrites previous version of returned_labels_FOR_ANALYSIS.csv 
df.to_csv(path +'/processing_labels/returned_labels_FOR_ANALYSIS.csv', index=False)

In [ ]:
#further cleanup
#rename columns
df.columns=['ID_allreviews_index','review','what_reviewed','sentiment',
             'effective_general','effective_stress','effective_sleep',
             'effective_anxiety','sideeffect','ProductName']

#ensure there are no dupes and set index to match index of allreviews.csv
df=df.drop_duplicates()
#df= df.set_index('ID_allreviews_index')

#drop product name - name was needed to give context to labelers
df=df.drop(['ProductName'], axis=1)

#insert numeric values for labels
df['what_reviewed']=df['what_reviewed'].map({'0 - Just the Service':0,'1 - Just the Product or unknown':1,'2 - Both':2})
df['sentiment']=df['sentiment'].map({'0=Neutral':0,'1=Positive':1,'2=Negative1':2})
df['effective_general']=df['effective_general'].map({'3 - review does not say':3,'1 - YES':1,'2 - NO does not work':2})
df['effective_stress']=df['effective_stress'].map({'3 - review does not say':3,'1 - YES':1,'2 - NO does not work':2})
df['effective_sleep']=df['effective_sleep'].map({'3 - review does not say':3,'1 - YES':1,'2 - NO does not work':2})
df['effective_anxiety']=df['effective_anxiety'].map({'3 - review does not say':3,'1 - YES':1,'2 - NO does not work':2})
df['sideeffect']=df['sideeffect'].map({'0 - NO':0,'1 - YES':1})

#cleanup - replace null values
df['what_reviewed']=df['what_reviewed'].fillna(1).astype(np.int64)
df['sentiment']=df['sentiment'].fillna(0).astype(np.int64)
df['effective_general']=df['effective_general'].fillna(3).astype(np.int64)
df['effective_stress']=df['effective_stress'].fillna(3).astype(np.int64)
df['effective_sleep']=df['effective_sleep'].fillna(3).astype(np.int64)
df['effective_anxiety']=df['effective_anxiety'].fillna(3).astype(np.int64)
df['sideeffect']=df['sideeffect'].fillna(0).astype(np.int64)


In [ ]:
#save file - note this overwrites previous version of codedlabels_FOR_MODEL.csv 
df.to_csv(path +'/processing_labels/codedlabels_FOR_MODEL.csv', index=False)